In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import unquote

# Set up the Chrome driver
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

# URL of the page containing the restaurant listings
url = "https://www.yellowpages.ca/search/si/1/Restaurants/Toronto+ON"

driver.get(url)

# Find all the result elements
results = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "listing__content__wrapper")))

# Initialize lists to store scraped data
names = []
addresses = []
phones = []
restaurant_websites = []

for result in results:
    name_element = result.find_element(By.CLASS_NAME, "listing__name--link")
    address_element = result.find_element(By.CLASS_NAME, "listing__address--full")
    
    try:
        phone_element = result.find_element(By.CLASS_NAME, "jsMlrMenu")
        phone_number = phone_element.get_attribute("data-phone")
    except:
        phone_number = "N/A"
    
    try:
        website_element = result.find_element(By.CSS_SELECTOR, "li.mlr__item.mlr__item--website a")
        restaurant_website_url = website_element.get_attribute("href")
        website_url_parts = restaurant_website_url.split("?redirect=")
        if len(website_url_parts) > 1:
            website_url_encoded = website_url_parts[1]
            restaurant_website_url = unquote(website_url_encoded).replace("http://", "").replace("https://", "")
    except:
        restaurant_website_url = "N/A"

    names.append(name_element.text.strip())
    addresses.append(address_element.text.strip())
    phones.append(phone_number)
    restaurant_websites.append(restaurant_website_url)

driver.quit()

# Create a pandas DataFrame
data = {
    "Name": names,
    "Address": addresses,
    "Phone": phones,
    "Restaurant Website": restaurant_websites
}

df = pd.DataFrame(data)

# Extract actual restaurant website URLs
def extract_actual_website_url(url):
    redirect_param = "redirect="
    if redirect_param in url:
        return url.split(redirect_param)[1]
    return url

df["Restaurant Website"] = df["Restaurant Website"].apply(extract_actual_website_url)

# Print the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv("restaurant_data.csv", index=False)
